In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import glob
import datetime as dt
import multiprocessing as mp
from datetime import datetime
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas import Series
from tqdm import tqdm, tqdm_notebook
import time
import sys
import matplotlib.pyplot as plt




from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

In [3]:
cd gdrive/My Drive/TFM/

/content/gdrive/My Drive/TFM


#Loading of already sampled DOLLAR BARS

In [24]:
# raw trade data from https://public.bitmex.com/?prefix=data/trade/ 
Dollar_bars = pd.DataFrame()
for i,file in enumerate(glob.glob("data/bars/new_features/*.csv")):
  if i == 0:
    Dollar_bars = Dollar_bars.append(pd.read_csv(file))
    print('Percentge of files already Loaded:',round((i/len(glob.glob("data/bars/new_features/*.csv")))*100,1), '%. There are', len(glob.glob("data/bars/new_features/*.csv"))-i, "files left", end='')
  else:
    Dollar_bars = Dollar_bars.append(pd.read_csv(file))
    print('\r Percentge of files already Loaded:',round((i/len(glob.glob("data/bars/new_features/*.csv")))*100,1), '%. There are', len(glob.glob("data/bars/new_features/*.csv"))-i, "files left",end='', flush=True)

Percentge of files already Loaded: 0.0 %. There are 1 files left

In [25]:
Dollar_bars.drop(columns=['Unnamed: 0','timestamp.1'],inplace=True)
# timestamp parsing
Dollar_bars['timestamp'] = Dollar_bars.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%d %H:%M:%S.%f"))

Dollar_bars.index = Dollar_bars['timestamp']
Dollar_bars


,timestamp,open,high,low,close,grossValue,homeNotional,foreignNotional,tweet_count,tweet_count2,Google_trend1,Google_trend2
timestamp,,,,,,,,,,,,
2019-12-22 00:00:02.857,2019-12-22 00:00:02.857,126.95,127.60,126.90,127.00,133995088605,75212.579042,9.563766e+06,379.0,172.0,42.0,55.0
2019-12-22 06:51:01.172,2019-12-22 06:51:01.172,127.00,128.80,126.95,128.45,133739653760,74811.562495,9.556766e+06,376.0,204.0,51.0,70.0
2019-12-22 07:42:27.825,2019-12-22 07:42:27.825,128.45,129.50,128.25,129.50,133638036400,74270.069115,9.573891e+06,609.0,359.0,56.0,58.0
2019-12-22 07:57:26.330,2019-12-22 07:57:26.330,129.50,131.45,129.40,130.45,132643814650,73244.938417,9.539001e+06,609.0,359.0,56.0,58.0
2019-12-22 08:02:58.674,2019-12-22 08:02:58.674,130.45,130.55,128.85,129.00,133612879225,73900.574533,9.580523e+06,737.0,211.0,55.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-23 22:32:07.348,2021-05-23 22:32:07.348,2091.35,2115.05,2065.90,2065.95,27670542430,4552.292856,9.452077e+06,3903.0,2226.0,74.0,41.0
2021-05-23 22:50:54.588,2021-05-23 22:50:54.588,2065.95,2075.10,2060.00,2065.00,28421512785,4657.210306,9.631046e+06,3903.0,2226.0,74.0,41.0
2021-05-23 22:53:52.767,2021-05-23 22:53:52.767,2065.00,2104.25,2065.00,2090.00,28385242445,4636.918646,9.626955e+06,3903.0,2226.0,74.0,41.0


#CLASS DEFINITION

##MULTIPROCESSING

In [26]:
class MultiProcessingFunctions:
	""" This static functions in this class enable multi-processing"""
	def __init__(self):
		pass

	@staticmethod
	def lin_parts(num_atoms, num_threads):
		""" This function partitions a list of atoms in subsets (molecules) of equal size.
		An atom is a set of indivisible set of tasks.
		"""

		# partition of atoms with a single loop
		parts = np.linspace(0, num_atoms, min(num_threads, num_atoms) + 1)
		parts = np.ceil(parts).astype(int)
		return parts

	@staticmethod
	def nested_parts(num_atoms, num_threads, upper_triangle=False):
		""" This function enables parallelization of nested loops.
		"""
		# partition of atoms with an inner loop
		parts = []
		num_threads_ = min(num_threads, num_atoms)

		for num in range(num_threads_):
			part = 1 + 4 * (parts[-1] ** 2 + parts[-1] + num_atoms * (num_atoms + 1.) / num_threads_)
			part = (-1 + part ** .5) / 2.
			parts.append(part)

		parts = np.round(parts).astype(int)

		if upper_triangle:  # the first rows are heaviest
			parts = np.cumsum(np.diff(parts)[::-1])
			parts = np.append(np.array([0]), parts)
		return parts

	@staticmethod
	def mp_pandas_obj(func, pd_obj, num_threads=24, mp_batches=1, lin_mols=True, **kargs):
		"""	
		:param func: (string) function to be parallelized
		:param pd_obj: (vector) Element 0, is name of argument used to pass the molecule;
						Element 1, is the list of atoms to be grouped into a molecule
		:param num_threads: (int) number of threads
		:param mp_batches: (int) number of batches
		:param lin_mols: (bool) Tells if the method should use linear or nested partitioning
		:param kargs: (var args)
		:return: (data frame) of results
		"""

		if lin_mols:
			parts = MultiProcessingFunctions.lin_parts(len(pd_obj[1]), num_threads * mp_batches)
		else:
			parts = MultiProcessingFunctions.nested_parts(len(pd_obj[1]), num_threads * mp_batches)

		jobs = []
		for i in range(1, len(parts)):
			job = {pd_obj[0]: pd_obj[1][parts[i - 1]:parts[i]], 'func': func}
			job.update(kargs)
			jobs.append(job)

		if num_threads == 1:
			out = MultiProcessingFunctions.process_jobs_(jobs)
		else:
			out = MultiProcessingFunctions.process_jobs(jobs, num_threads=num_threads)

		if isinstance(out[0], pd.DataFrame):
			df0 = pd.DataFrame()
		elif isinstance(out[0], pd.Series):
			df0 = pd.Series()
		else:
			return out

		for i in out:
			df0 = df0.append(i)

		df0 = df0.sort_index()
		return df0

	@staticmethod
	def process_jobs_(jobs):
		""" Run jobs sequentially, for debugging """
		out = []
		for job in jobs:
			out_ = MultiProcessingFunctions.expand_call(job)
			out.append(out_)
		return out

	@staticmethod
	def expand_call(kargs):
		""" Expand the arguments of a callback function, kargs['func'] """
		func = kargs['func']
		del kargs['func']
		out = func(**kargs)
		return out

	@staticmethod
	def report_progress(job_num, num_jobs, time0, task):
		# Report progress as asynch jobs are completed

		msg = [float(job_num) / num_jobs, (time.time() - time0)/60.]
		msg.append(msg[1] * (1/msg[0] - 1))
		time_stamp = str(dt.datetime.fromtimestamp(time.time()))

		msg = time_stamp + ' ' + str(round(msg[0]*100, 2)) + '% '+task+' done after ' + \
			str(round(msg[1], 2)) + ' minutes. Remaining ' + str(round(msg[2], 2)) + ' minutes.'

		if job_num < num_jobs:
			sys.stderr.write(msg+'\r')
		else:
			sys.stderr.write(msg+'\n')

		return

	@staticmethod
	def process_jobs(jobs, task=None, num_threads=24):
		""" Run in parallel. jobs must contain a 'func' callback, for expand_call"""

		if task is None:
			task = jobs[0]['func'].__name__

		pool = mp.Pool(processes=num_threads)
		# outputs, out, time0 = pool.imap_unordered(MultiProcessingFunctions.expand_call,jobs),[],time.time()
		outputs = pool.imap_unordered(MultiProcessingFunctions.expand_call, jobs)
		out = []
		time0 = time.time()

		# Process asyn output, report progress
		for i, out_ in enumerate(outputs, 1):
			out.append(out_)
			MultiProcessingFunctions.report_progress(i, len(jobs), time0, task)

		pool.close()
		pool.join()  # this is needed to prevent memory leaks
		return out

##VOLATILITY, CUSUM FILTER AND TRIPLE BARRIER

In [27]:
#SNIPPET 3.1 DAILY VOLATILITY ESTIMATES

def get_daily_vol(close, lookback=100):
    """
    :param close: (data frame) Closing prices
    :param lookback: (int) lookback period to compute volatility
    :return: (series) of daily volatility value
    """
    print('Calculating daily volatility for dynamic thresholds')
    
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0 > 0]
    df0 = (pd.Series(close.index[df0 - 1], index=close.index[close.shape[0] - df0.shape[0]:]))
        
    df0 = close.loc[df0.index] / close.loc[df0.values].values - 1  # daily returns
    df0 = df0.ewm(span=lookback).std()
    return df0

In [28]:
def get_t_events(raw_price, threshold):
    """
    :param raw_price: (series) of close prices.
    :param threshold: (float) when the abs(change) is larger than the threshold, the
    function captures it as an event.
    :return: (datetime index vector) vector of datetimes when the events occurred. This is used later to sample.
    """
    print('Applying Symmetric CUSUM filter.')

    t_events = []
    s_pos = 0
    s_neg = 0

    # log returns
    diff = np.log(raw_price).diff().dropna()

    # Get event time stamps for the entire series
    for i in tqdm(diff.index[1:]):
        pos = float(s_pos + diff.loc[i])
        neg = float(s_neg + diff.loc[i])
        s_pos = max(0.0, pos)
        s_neg = min(0.0, neg)

        if s_neg < -threshold:
            s_neg = 0
            t_events.append(i)

        elif s_pos > threshold:
            s_pos = 0
            t_events.append(i)

    event_timestamps = pd.DatetimeIndex(t_events)
    return event_timestamps

In [29]:
# SNIPPET 3.4 ADDING A VERTICAL BARRIER

def add_vertical_barrier(t_events, close, num_days=10):
    """
    :param t_events: (series) series of events (symmetric CUSUM filter)
    :param close: (series) close prices
    :param num_days: (int) maximum number of days a trade can be active
    :return: (series) timestamps of vertical barriers
    """
    
    t1 = close.index.searchsorted(t_events + pd.Timedelta(days=num_days))
    t1 = t1[t1 < close.shape[0]]
    t1 = pd.Series(close.index[t1], index=t_events[:t1.shape[0]])  # NaNs at end
    return t1

In [30]:
#SNIPPET 3.2 TRIPLE-BARRIER LABELING METHOD

def apply_pt_sl_on_t1(close, events, pt_sl, molecule):
    """
    :param close: (series) close prices
    :param events: (series) of indices that signify "events" 
    :param pt_sl: (array) element 0, indicates the profit taking level; 
                          element 1 is stop loss level
    :param molecule: (an array) a set of datetime index values for processing
    :return: (dataframe) timestamps at which each barrier was touched
    """
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_ = events.loc[molecule]
    out = events_[['t1']].copy(deep=True)
    if pt_sl[0] > 0:
        pt = pt_sl[0] * events_['trgt']
    else:
        pt = pd.Series(index=events.index)  # NaNs

    if pt_sl[1] > 0:
        sl = -pt_sl[1] * events_['trgt']
    else:
        sl = pd.Series(index=events.index)  # NaNs

    for loc, t1 in events_['t1'].fillna(close.index[-1]).iteritems():
        df0 = close[loc:t1]  # path prices
        df0 = (df0 / close[loc] - 1) * events_.at[loc, 'side']  # path returns
        out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()  # earliest stop loss
        out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()  # earliest profit taking

    return out

In [31]:
# SNIPPET 3.3 GETTING THE TIME OF FIRST TOUCH

def get_events(close, t_events, pt_sl, target, min_ret, num_threads, 
              vertical_barrier_times=False, side=None):
    """
    :param close: (series) Close prices
    :param t_events: (series) of t_events. 
                     These are timestamps that will seed every triple barrier.
    :param pt_sl: (2 element array) element 0, indicates the profit taking percentage; 
                  element 1 is stop loss percentage.
                  A non-negative float that sets the width of the two barriers. 
                  A 0 value means that the respective horizontal barrier will be disabled.
    :param target: (series) of values that are used (in conjunction with pt_sl)
                   to determine the width of the barrier.
    :param min_ret: (float) The minimum target return required for running a triple barrier search.
    :param num_threads: (int) The number of threads concurrently used by the function.
    :param vertical_barrier_times: (series) A pandas series with the timestamps of the vertical barriers.
    :param side: (series) Side of the bet (long/short) as decided by the primary model
    :return: (data frame) of events
            -events.index is event's starttime
            -events['t1'] is event's endtime
            -events['trgt'] is event's target
            -events['side'] (optional) implies the algo's position side
    """

    # 1) Get target
    target = target.loc[target.index.intersection(t_events)]
    target = target[target > min_ret]  # min_ret

    # 2) Get vertical barrier (max holding period)
    if vertical_barrier_times is False:
        vertical_barrier_times = pd.Series(pd.NaT, index=t_events)

    # 3) Form events object, apply stop loss on vertical barrier
    if side is None:
        side_ = pd.Series(1., index=target.index)
        pt_sl_ = [pt_sl[0], pt_sl[0]]
    else:
        side_ = side.loc[target.index]
        pt_sl_ = pt_sl[:2]

    events = pd.concat({'t1': vertical_barrier_times, 'trgt': target, 'side': side_},
                        axis=1)
    events = events.dropna(subset=['trgt'])

    # Apply Triple Barrier
    df0 = MultiProcessingFunctions.mp_pandas_obj(func=apply_pt_sl_on_t1,
                                                 pd_obj=('molecule', events.index),
                                                 num_threads=num_threads,
                                                 close=close,
                                                 events=events,
                                                 pt_sl=pt_sl_)

    events['t1'] = df0.dropna(how='all').min(axis=1)  # pd.min ignores nan

    if side is None:
        events = events.drop('side', axis=1)

    return events

In [32]:
def barrier_touched(out_df):
    """
    :param out_df: (DataFrame) containing the returns and target
    :return: (DataFrame) containing returns, target, and labels
    """
    store = []
    for i in np.arange(len(out_df)):
        date_time = out_df.index[i]
        ret = out_df.loc[date_time, 'ret']
        target = out_df.loc[date_time, 'trgt']

        if ret > 0.0 and ret > target:
            # Top barrier reached
            store.append(1)
        elif ret < 0.0 and ret < -target:
            # Bottom barrier reached
            store.append(-1)
        else:
            # Vertical barrier reached
            store.append(0)

    out_df['bin'] = store

    return out_df

In [33]:
# SNIPPET 3.5 LABELING FOR SIDE AND SIZE

def get_bins(triple_barrier_events, close):
    """
    :param triple_barrier_events: (data frame)
                -events.index is event's starttime
                -events['t1'] is event's endtime
                -events['trgt'] is event's target
                -events['side'] (optional) implies the algo's position side
                Case 1: ('side' not in events): bin in (-1,1) <-label by price action
                Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    :param close: (series) close prices
    :return: (data frame) of meta-labeled events
    """

    # 1) Align prices with their respective events
    events_ = triple_barrier_events.dropna(subset=['t1'])
    prices = events_.index.union(events_['t1'].values)
    prices = prices.drop_duplicates()
    prices = close.reindex(prices, method='bfill')
    
    # 2) Create out DataFrame
    out_df = pd.DataFrame(index=events_.index)
    # Need to take the log returns, else your results will be skewed for short positions
    out_df['ret'] = np.log(prices.loc[events_['t1'].values].values) - np.log(prices.loc[events_.index])
    out_df['trgt'] = events_['trgt']

    # Meta labeling: Events that were correct will have pos returns
    if 'side' in events_:
        out_df['ret'] = out_df['ret'] * events_['side']  # meta-labeling

    # Added code: label 0 when vertical barrier reached
    out_df = barrier_touched(out_df)

    # Meta labeling: label incorrect events with a 0
    if 'side' in events_:
        out_df.loc[out_df['ret'] <= 0, 'bin'] = 0
    
    # Transform the log returns back to normal returns.
    out_df['ret'] = np.exp(out_df['ret']) - 1
    
    # Add the side to the output. This is useful for when a meta label model must be fit
    tb_cols = triple_barrier_events.columns
    if 'side' in tb_cols:
        out_df['side'] = triple_barrier_events['side']
        
    out_df

    return out_df

##PRIMARY MODEL AUX METHODS AND CLASSES

In [34]:
# The function to add a certain number of columns
def adder(Data, times):
    
    for i in range(1, times + 1):
    
        z = np.zeros((len(Data), 1), dtype = float)
        Data = np.append(Data, z, axis = 1)        
    return Data
# The function to deleter a certain number of columns
def deleter(Data, index, times):
    
    for i in range(1, times + 1):
    
        Data = np.delete(Data, index, axis = 1)               
    
    return Data
# The function to delete a certain number of rows from the beginning
def jump(Data, jump):
    
    Data = Data[jump:, ]
    
    return Data

In [35]:
def stochastic(Data, lookback, high, low, close, where, genre = 'High-Low'):
        
    # Adding a column
    Data = adder(Data, 1)
    
    if genre == 'High-Low':
        
        for i in range(len(Data)):
            
            try:
                Data[i, where] = (Data[i, close] - min(Data[i - lookback + 1:i + 1, low])) / (max(Data[i - lookback + 1:i + 1, high]) - min(Data[i - lookback + 1:i + 1, low]))
            
            except ValueError:
                pass
            
    if genre == 'Normalization':
        
        for i in range(len(Data)):
            
            try:
                Data[i, where] = (Data[i, close] - min(Data[i - lookback + 1:i + 1, close])) / (max(Data[i - lookback + 1:i + 1, close]) - min(Data[i - lookback + 1:i + 1, close]))
            
            except ValueError:
                pass
            
    Data[:, where] = Data[:, where] * 100  
    Data = jump(Data, lookback)
    return Data

##Average uniqueness and weights by returns

In [36]:
def mpNumCoEvents(closeIdx,t1,molecule):
  #Compute the number of concurrent events per bar.
  #+molecule[0] is the date of the first event on which the weight will be computed
  #+molecule[-1] is the date of the last event on which the weight will be computed
  #Any event that starts before t1[molecule].max() impacts the count.
  #1) find events that span the period [molecule[0],molecule[-1]]
  t1=t1.fillna(closeIdx[-1]) # unclosed events still must impact other weights
  t1=t1[t1>=molecule[0]] # events that end at or after molecule[0] 
  t1=t1.loc[:t1[molecule].max()]# events that start at or before t1[molecule].max()
  #2) count events spanning a bar
  iloc=closeIdx.searchsorted(np.array([t1.index[0],t1.max()]))
  count=pd.Series(0,index=closeIdx[iloc[0]:iloc[1]+1])
  for tIn,tOut in t1.iteritems():count.loc[tIn:tOut]+=1.
  return count.loc[molecule[0]:t1[molecule].max()]  

In [37]:
def mpSampleTW(t1,numCoEvents,molecule):
    # Derive average uniqueness over the event's lifespan
    wght=pd.Series(index=molecule)
    for tIn,tOut in t1.loc[wght.index].iteritems():
      wght.loc[tIn]=(1./numCoEvents.loc[tIn:tOut]).mean()
    return wght

In [38]:
def getIndMatrix(barIx,t1):
  # Get indicator matrix
  indM=pd.DataFrame(0,index=barIx,columns=range(t1.shape[0]))
  for i,(t0,t1) in enumerate(t1.iteritems()):indM.loc[t0:t1,i]=1.
  return indM

In [39]:
def getAvgUniqueness(indM):
  # Average uniqueness from indicator matrix
  c=indM.sum(axis=1) # concurrency
  u=indM.div(c,axis=0) # uniqueness
  avgU=u[u>0].mean() # average uniqueness
  return avgU

In [40]:
def seqBootstrap(indM,sLength=None):
  # Generate a sample via sequential bootstrap
  if sLength is None:sLength=indM.shape[1]
  phi=[]
  while len(phi)<sLength:
    avgU=pd.Series()
    for i in indM:
      indM_=indM[phi+[i]] # reduce indM
      avgU.loc[i]=getAvgUniqueness(indM_).iloc[-1]
    prob=avgU/avgU.sum() # draw prob
    phi+=[np.random.choice(indM.columns,p=prob)]
  return phi

In [41]:
def main():
  t1=triple_barrier_events['t1'] # t0,t1 for each feature obs
  barIx=triple_barrier_events.index # index of bars
  indM=getIndMatrix(triple_barrier_events.index,triple_barrier_events['t1'])
  phi = np.random.choice(indM.columns,size=indM.shape[1])
  print(phi)
  print('Standard uniqueness:', getAvgUniqueness(indM[phi]).mean())
  phi = seqBootstrap(indM)
  print(phi)
  print('Sequential uniqueness:',getAvgUniqueness(indM[phi]).mean())
  return

In [42]:
def mpSampleW(t1,numCoEvents,close,molecule):
  # Derive sample weight by return attribution
  ret=np.log(close).diff() # log-returns, so that they are additive
  wght=pd.Series(index=molecule)
  for tIn,tOut in t1.loc[wght.index].iteritems():
    wght.loc[tIn]=(ret.loc[tIn:tOut]/numCoEvents.loc[tIn:tOut]).sum()
  return wght.abs()

##Time decay calc

In [43]:
def getTimeDecay(tW,clfLastW=1.):
  # apply piecewise-linear decay to observed uniqueness (tW)
  # newest observation gets weight=1, oldest observation gets weight=clfLastW
  # if clfLastW = 1.0 means that there is no time decay.
  # if 0.0 < clfLastW < 1.0 means that weights decay linearly over time,
  #   but every observation still receives a strictly positive weight, regardless of how old.
  # if clfLastW = 0.0 means that weights converge linearly to zero, as they become older.
  # if clfLastW < 0.0 means that the oldest portion cT of the observations receive zero weight
  #(i.e., they are erased from memory).
 clfW=tW.sort_index().cumsum()
 if clfLastW>=0:slope=(1.-clfLastW)/clfW.iloc[-1]
 else:slope=1./((clfLastW+1)*clfW.iloc[-1])
 const=1.-slope*clfW.iloc[-1]
 clfW=const+slope*clfW
 clfW[clfW<0]=0
 print(const,slope)
 return clfW

#PRIMARY MODEL

##MEAN REVERTING STRATEGY BASED ON STOCHASTIC INDICATOR

In [44]:
# Indicator Parameters
lookback = 31
upper_barrier = 95
lower_barrier = 5
my_data = adder(Dollar_bars[["open", "high", "low", "close"]].to_numpy(), 10)
my_data = stochastic(my_data, lookback, 1, 2, 3, 4)

Dollar_bars['stochastic'] = np.append([np.nan]*31,my_data[:, 4])


In [45]:
# compute sides
count_longs = np.zeros((len(Dollar_bars),1), dtype=bool)
count_shorts = np.zeros((len(Dollar_bars),1), dtype=bool)


aux = Dollar_bars['stochastic'][0]


for idx, val in enumerate(Dollar_bars['stochastic']):
  if val >= 5 and aux < 5:
    #print("realizaria un long en el index", idx)
    count_longs[idx] = True;
  if val <= 95 and aux > 95:
    #print("realizaria un short en el index", idx)
    count_shorts[idx] = True;
  aux = val


In [46]:
# compute sides
Dollar_bars['side'] = np.nan

long_signals = Dollar_bars.loc[count_longs].index
short_signals = Dollar_bars.loc[count_shorts].index
Dollar_bars.loc[long_signals, 'side'] = 1
Dollar_bars.loc[short_signals, 'side'] = -1

print(Dollar_bars.side.value_counts())

# Remove Look ahead biase by lagging the signal
Dollar_bars['side'] = Dollar_bars['side'].shift(1)

# Drop the NaN values from our data set
Dollar_bars.dropna(axis=0, how='any', inplace=True)  

-1.0    1283
 1.0     817
Name: side, dtype: int64


In [47]:
# determining daily volatility using the last 50 days
daily_vol = get_daily_vol(close=Dollar_bars['close'], lookback=50)

Calculating daily volatility for dynamic thresholds


In [48]:
close = Dollar_bars['close']

# creating our event triggers using the CUSUM filter 
cusum_events = get_t_events(Dollar_bars['close'], threshold=daily_vol.mean())

# adding vertical barriers with a half day expiration window
vertical_barriers = add_vertical_barrier(t_events=cusum_events,
                                         close=close, num_days=0.5)

# determining timestamps of first touch   

pt_sl = [1, 2] # setting profit-take and stop-loss at 1% and 2%
min_ret = 0.005 # setting a minimum return of 0.5% 

triple_barrier_events = get_events(close=close,
                                  t_events=cusum_events,
                                  pt_sl=pt_sl,
                                  target=daily_vol,
                                  min_ret=min_ret,
                                  num_threads=2,
                                  vertical_barrier_times=vertical_barriers,
                                  side=Dollar_bars['side']
                                   )

Applying Symmetric CUSUM filter.


100%|██████████| 2098/2098 [00:00<00:00, 19301.88it/s]
2021-12-14 11:40:25.927181 100.0% apply_pt_sl_on_t1 done after 0.01 minutes. Remaining 0.0 minutes.


In [54]:
labels = get_bins(triple_barrier_events, Dollar_bars['close'])
labels.side.value_counts()  

-1.0    184
 1.0    135
Name: side, dtype: int64

In [55]:
numCoEvents = MultiProcessingFunctions.mp_pandas_obj(func=mpNumCoEvents,
                                                     pd_obj= ('molecule',triple_barrier_events.index),
                                                     num_threads=24,
                                                     closeIdx=close.index,
                                                     t1=triple_barrier_events['t1']
                                                     )

2021-12-14 11:40:49.212852 100.0% mpNumCoEvents done after 0.01 minutes. Remaining 0.0 minutes.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [56]:
numCoEvents = numCoEvents.loc[~numCoEvents.index.duplicated(keep='last')]

In [57]:
numCoEvents = numCoEvents.reindex(close.index).fillna(0)

In [58]:
print(numCoEvents.value_counts())
window = 50
labels['ret'].ewm(span=window).std()

1.0    1052
0.0     759
2.0     217
3.0      60
4.0      10
5.0       2
dtype: int64


2019-12-29 17:29:42.425         NaN
2020-01-05 02:21:43.122    0.037442
2020-01-08 01:04:07.761    0.032133
2020-01-14 04:04:19.490    0.053174
2020-01-14 17:49:57.387    0.049157
                             ...   
2021-05-21 01:48:26.038    0.115286
2021-05-21 08:15:37.398    0.118627
2021-05-21 19:50:46.707    0.116609
2021-05-23 09:14:16.219    0.114338
2021-05-23 16:21:04.716    0.115600
Name: ret, Length: 319, dtype: float64

In [59]:
fig = make_subplots(rows=1, cols=1,specs=[[{"secondary_y": True}]])
# Create figure with secondary y-axis

fig.add_trace(go.Scatter(
    x = numCoEvents.index,
    y = numCoEvents,
    #y=labels['ret'].ewm(span=window).std(),
    name="Number of concurrent samples",
    mode = 'lines',
    textfont_family="Arial_Black"),
    row= 1 , 
    col= 1 )
fig.add_trace(go.Scatter(
    x=labels['ret'].ewm(span=window).std().index,
    y=labels['ret'].ewm(span=window).std(),
    name="exponentially weighted moving standard deviation of returns",
    mode = 'lines',
    textfont_family="Arial_Black"),
    secondary_y=True,
    row= 1 , 
    col= 1 )

In sklearn, the sklearn.ensemble.BaggingClassifier class accepts an argument max_samples, which can be set to:

#max_samples=out['tW'].mean().

In this way, we enforce that the in-bag observations are not sampled at a frequency much higher than their uniqueness.

In [ ]:
fig = make_subplots(rows=1, cols=1)


fig.add_trace(go.Scatter(
    x=labels.index,
    y=labels['tW'],
    name="Average concurrent labels",
    mode = 'lines',
    textfont_family="Arial_Black"),
    row= 1 , 
    col= 1 )


In [60]:
labels['w'] = MultiProcessingFunctions.mp_pandas_obj(func= mpSampleW,
                                                   pd_obj = ('molecule',triple_barrier_events.index),
                                                   num_threads = 24,
                                                   t1 = triple_barrier_events['t1'],
                                                   numCoEvents = numCoEvents,
                                                   close = close)
labels['w']*=labels['w'].shape[0]/labels['w'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: Deprecat

In [63]:
#CALLING SNIPPET 4.2 AVERAGE UNIQUENESS OF EACH LABEL
labels['tW'] = MultiProcessingFunctions.mp_pandas_obj(func= mpSampleTW,
                                                   pd_obj = ('molecule',triple_barrier_events.index),
                                                   num_threads = mp.cpu_count(),
                                                   t1 = triple_barrier_events['t1'],
                                                   numCoEvents = numCoEvents)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

2021-12-14 11:42:11.797033 100.0% mpSampleTW done after 0.0 minutes. Remaining 0.0 minutes.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [64]:
print('range for tW (observed uniqueness score of a label) feature:',labels['tW'].min(),'to: ',labels['tW'].max())
print('range for w (sample weights) feature:',labels['w'].min(),'to: ',labels['w'].max())


range for tW (observed uniqueness score of a label) feature: 0.21666666666666667 to:  1.0
range for w (sample weights) feature: 0.0014881173486447682 to:  4.590280602516388


In [73]:
cs = {1.0,0.75,0.5,-0.25,-0.5}
getTimeDecay(labels['tW'],0.75)

0.75 0.0009640344298788259


2019-12-29 17:29:42.425    0.750964
2020-01-05 02:21:43.122    0.751928
2020-01-08 01:04:07.761    0.752892
2020-01-14 04:04:19.490    0.753856
2020-01-14 17:49:57.387    0.754820
                             ...   
2021-05-21 01:48:26.038    0.997846
2021-05-21 08:15:37.398    0.998512
2021-05-21 19:50:46.707    0.999235
2021-05-23 09:14:16.219    0.999705
2021-05-23 16:21:04.716    1.000000
Name: tW, Length: 319, dtype: float64

In [74]:
cs = [1, 0.75, 0.5, -0.25, -0.5]
dfac_1 = getTimeDecay(labels['tW'],1.0)
dfac_75 = getTimeDecay(labels['tW'],0.75)
dfac_5 = getTimeDecay(labels['tW'],0.5)
dfac_ = getTimeDecay(labels['tW'],0.0)
dfac_25_neg = getTimeDecay(labels['tW'],-0.25)
dfac_5_neg = getTimeDecay(labels['tW'],-0.5)


1.0 0.0
0.75 0.0009640344298788259
0.5 0.0019280688597576518
0.0 0.0038561377195153036
-0.33333333333333326 0.0051415169593537376
-1.0 0.007712275439030607


In [75]:
fig = make_subplots(rows=1, cols=1)


fig.add_trace(go.Scatter(
    x=dfac_1.index,
    y=dfac_1,
    name="Decay factor = 1.0",
    mode = 'lines',
    textfont_family="Arial_Black"),
    row= 1 , 
    col= 1 )
fig.add_trace(go.Scatter(
    x=dfac_75.index,
    y=dfac_75,
    name="Decay factor = 0.75",
    mode = 'lines',
    textfont_family="Arial_Black"),
    row= 1 , 
    col= 1 )
fig.add_trace(go.Scatter(
    x=dfac_5.index,
    y=dfac_5,
    name="Decay factor = 0.5",
    mode = 'lines',
    textfont_family="Arial_Black"),
    row= 1 , 
    col= 1 )
fig.add_trace(go.Scatter(
    x=dfac_.index,
    y=dfac_,
    name="Decay factor = 0.0",
    mode = 'lines',
    textfont_family="Arial_Black"),
    row= 1 , 
    col= 1 )
fig.add_trace(go.Scatter(
    x=dfac_25_neg.index,
    y=dfac_25_neg,
    name="Decay factor = -0.25",
    mode = 'lines',
    textfont_family="Arial_Black"),
    row= 1 , 
    col= 1 )
fig.add_trace(go.Scatter(
    x=dfac_5_neg.index,
    y=dfac_5_neg,
    name="Decay factor = -0.5",
    mode = 'lines',
    textfont_family="Arial_Black"),
    row= 1 , 
    col= 1 )


In [76]:
labels['decayWeights'] = getTimeDecay(labels['tW'],0.5)
labels

0.5 0.0019280688597576518


,ret,trgt,bin,side,w,tW,decayWeights
2019-12-29 17:29:42.425,0.044566,0.012139,1,-1.0,0.527131,1.000000,0.501928
2020-01-05 02:21:43.122,-0.008385,0.032676,0,-1.0,0.594249,1.000000,0.503856
2020-01-08 01:04:07.761,0.049445,0.028223,1,-1.0,0.751997,1.000000,0.505784
2020-01-14 04:04:19.490,-0.062382,0.031907,0,-1.0,2.558999,1.000000,0.507712
2020-01-14 17:49:57.387,-0.035484,0.047432,0,-1.0,1.652547,1.000000,0.509640
...,...,...,...,...,...,...,...
2021-05-21 01:48:26.038,0.112456,0.202989,0,-1.0,0.897340,0.388889,0.995692
2021-05-21 08:15:37.398,-0.159263,0.200009,0,1.0,3.009998,0.690476,0.997024
2021-05-21 19:50:46.707,-0.027175,0.181426,0,1.0,1.900339,0.750000,0.998470
2021-05-23 09:14:16.219,0.000674,0.168070,0,1.0,1.924435,0.488095,0.999411
